In [1]:
%load_ext autoreload
%autoreload 2

import os 
import sys
import skimage.draw
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings("ignore")


In [2]:
train_dataset = "/allen/programs/celltypes/workgroups/em-connectomics/gayathrim/vesicle_detection/data/datasciencebowl2018/stage1_train"
val_dataset   = "/allen/programs/celltypes/workgroups/em-connectomics/gayathrim/vesicle_detection/data/datasciencebowl2018/stage1_val"
weights = "/allen/programs/celltypes/workgroups/em-connectomics/gayathrim/gits/Mask_RCNN/mask_rcnn_coco.h5"
weights = "/allen/programs/celltypes/workgroups/em-connectomics/gayathrim/vesicle_detection/vesicles_model_v1.h5"
logs = "/allen/programs/celltypes/workgroups/em-connectomics/gayathrim/vesicle_detection/logs"
command = "train"

In [3]:
# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR) 

from mrcnn import model as modellib, utils, visualize
from vesicles import VesicleDataset, VesicleConfig


# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")


# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")


def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a 
    central point to control graph sizes.

    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax


Using TensorFlow backend.


In [4]:
def train(model, dataset):
    """Train the model."""
    # Training dataset.
    dataset_train = VesicleDataset()
    dataset_train.load_vesicle(train_dataset)
    dataset_train.prepare()

    # Validation dataset
    dataset_val = VesicleDataset()
    dataset_val.load_vesicle(val_dataset)
    dataset_val.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")
    #model.train(dataset_train, dataset_val,
    #            learning_rate=config.LEARNING_RATE,
    #            epochs=30,
    #            layers='heads')

    # Training - Stage 2
    # Finetune layers from ResNet stage 4 and up
    print("Fine tune Resnet stage 4 and up")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=40,
                layers='4+')

    # Training - Stage 3
    # Fine tune all layers
    print("Fine tune all layers")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE / 10,
                epochs=70,
                layers='all')


def test(model, image_path):
    print("Running on {}".format(image_path))
    
    # Read image
    image = skimage.io.imread(image_path)

    # detect objects
    r = model.detect([image], verbose=1)[0]

    # Display results
    ax = get_ax(1)
    visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'],
                                ["Background", "IC"], r['scores'], ax=ax,
                                title="Predictions")



In [5]:
# Configurations
if command == "train":
    config = VesicleConfig()
else:
    class InferenceConfig(VesicleInferenceConfig):
        # Set batch size to 1 since we'll be running inference on
        # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
        GPU_COUNT = 1
        IMAGES_PER_GPU = 1
    config = InferenceConfig()
config.display()

# Create model
if command == "train":
    model = modellib.MaskRCNN(mode="training", config=config,
                              model_dir=logs)
else:
    model = modellib.MaskRCNN(mode="inference", config=config,
                              model_dir=logs)

weights_path = weights

# Load weights
print("Loading weights ", weights_path)
# Exclude the last layers because they require a matching
# number of classes
model.load_weights(weights_path, by_name=True, exclude=[
    "mrcnn_class_logits", "mrcnn_bbox_fc",
    "mrcnn_bbox", "mrcnn_mask"])

# Train or evaluate
if command == "train":
    train(model, train_dataset)
elif command == "test":
    test(model, image_path=image)
else:
    print("'{}' is not recognized. "
          "Use 'train' or 'test'".format(command))

W0327 18:24:40.540176 140287092823808 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0327 18:24:40.573536 140287092823808 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0327 18:24:40.607275 140287092823808 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0327 18:24:40.641825 140287092823808 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:1919: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

W0327 18:24:40.646507 140287092


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'mrcnn_mask_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

W0327 18:24:43.926426 140287092823808 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:2018: The name tf.image.resize_nearest_neighbor is deprecated. Please use tf.compat.v1.image.resize_nearest_neighbor instead.

W0327 18:24:44.706632 140287092823808 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/array_ops.py:1354: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0327 18:24:44.865433 140287092823808 deprecation_wrapper.py:119] From /allen/programs/celltypes/workgroups/em-connectomics/gayathrim/gits/Mask_RCNN/mrcnn/model.py:554: The name tf.random_shuffle is deprecated. Please use tf.random.shuffle instead.

W0327 18:24:44.944606 140287092823808 deprecation_wrapper.py:119] From /allen/programs/celltypes/workgroups/

Loading weights  /allen/programs/celltypes/workgroups/em-connectomics/gayathrim/vesicle_detection/vesicles_model_v1.h5

vesicle

vesicle
Training network heads
Fine tune Resnet stage 4 and up

Starting at epoch 0. LR=0.001

Checkpoint Path: /allen/programs/celltypes/workgroups/em-connectomics/gayathrim/vesicle_detection/logs/vesicles20200327T1824/mask_rcnn_vesicles_{epoch:04d}.h5
Selecting layers to train
res4a_branch2a         (Conv2D)
bn4a_branch2a          (BatchNorm)
res4a_branch2b         (Conv2D)
bn4a_branch2b          (BatchNorm)
res4a_branch2c         (Conv2D)
res4a_branch1          (Conv2D)
bn4a_branch2c          (BatchNorm)
bn4a_branch1           (BatchNorm)
res4b_branch2a         (Conv2D)
bn4b_branch2a          (BatchNorm)
res4b_branch2b         (Conv2D)
bn4b_branch2b          (BatchNorm)
res4b_branch2c         (Conv2D)
bn4b_branch2c          (BatchNorm)
res4c_branch2a         (Conv2D)
bn4c_branch2a          (BatchNorm)
res4c_branch2b         (Conv2D)
bn4c_branch2b          

W0327 18:24:57.768494 140287092823808 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0327 18:25:17.105041 140287092823808 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/callbacks.py:850: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0327 18:25:17.106225 140287092823808 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/callbacks.py:853: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch 1/40
50/50 [==============================] - 1500s 30s/step - loss: 1.1564 - rpn_class_loss: 0.0287 - rpn_bbox_loss: 0.1157 - mrcnn_class_loss: 0.3631 - mrcnn_bbox_loss: 0.2507 - mrcnn_mask_loss: 0.3982 - val_loss: 1.4359 - val_rpn_class_loss: 0.0694 - val_rpn_bbox_loss: 0.1998 - val_mrcnn_class_loss: 0.3736 - val_mrcnn_bbox_loss: 0.3245 - val_mrcnn_mask_loss: 0.4685
Epoch 2/40
50/50 [==============================] - 1306s 26s/step - loss: 0.6173 - rpn_class_loss: 0.0169 - rpn_bbox_loss: 0.0987 - mrcnn_class_loss: 0.2523 - mrcnn_bbox_loss: 0.0698 - mrcnn_mask_loss: 0.1795 - val_loss: 1.8506 - val_rpn_class_loss: 0.1980 - val_rpn_bbox_loss: 0.4166 - val_mrcnn_class_loss: 0.4787 - val_mrcnn_bbox_loss: 0.2864 - val_mrcnn_mask_loss: 0.4709
Epoch 3/40
50/50 [==============================] - 1314s 26s/step - loss: 0.9395 - rpn_class_loss: 0.0279 - rpn_bbox_loss: 0.2479 - mrcnn_class_loss: 0.2183 - mrcnn_bbox_loss: 0.1447 - mrcnn_mask_loss: 0.3008 - val_loss: 2.9127 - val_rpn_class_l